---

<div style="text-align: center; font-family: 'Arial', sans-serif; font-weight: bold; font-size: 30px;">
  Data Challenge DSW 2023
    <br>Senior Professional Category<br>
  <div style="text-align: center; font-family: 'Arial', sans-serif; font-size: 30px;">
  </div>
</div>
<div style="text-align: center; font-family: 'Arial', sans-serif; font-size: 30px;">
Semantic Matching for POS Product Mapping<br>
</div>

---

## 📑 Table of Contents

1. 🏢 **Business Context**  

2. ❗ **Problem Statement**  

3. 🎯 **Project Objectives**  

4. 📦 **Resources & Dataset**  

5. 🔧 **Methodology & Architecture**  

6. 🧹 **Data Overview, Cleaning & Preprocessing**  

7. 🔍 **Exploratory Data Analysis (EDA)**  

8. 🧠 **Feature Engineering**

9. 🧪 **Model Training & Evaluation**  

10. 📌 **Insights**

13. 📚 **References & Tools**  

---


1. **🏢 Business Context**

The fertilizer company maintains a structured product catalog based on SKU codes. However, many of their partners input inconsistent, free-text product names during Point-of-Sale (POS) transactions. These unstructured inputs create issues in maintaining product traceability, stock accuracy, and sales reporting.

2. **❗ Problem Statement**
   
The company needs a scalable solution to match free-text product names from POS systems to structured SKUs in the product catalog. Previous attempts using fuzzy matching proved slow, computationally expensive, and unable to capture semantic similarities or typos effectively.

3. **🎯 Project Objectives**
   
* Build a semantic matching system that maps noisy product names from POS to structured SKU names.
* Increase mapping accuracy without relying on exact string matches.
* Flag unmatched products as potential new SKUs.
* Ensure scalability and runtime efficiency for large datasets.

4. **📦 Resources & Dataset**

**Datasets Provided:**

**Product Catalog:**
* SKU
* Brand
* Type
* Formula

**POS Transactions:**
* Free-text Product Name

**Tools & Libraries:**
*Python, Pandas, FAISS, SentenceTransformers (all-MiniLM-L6-v2), scikit-learn

<div style="text-align: center;">
  <img src="https://www.mdpi.com/mathematics/mathematics-11-02700/article_deploy/html/images/mathematics-11-02700-g002-550.jpg" alt="semantic" width="250"/>
</div>

5. **🔧 Methodology & Architecture**

**Why Not Fuzzy Matching?**

Traditional fuzzy matching techniques like **Levenshtein Distance, Jaro-Winkler, or token set ratio** (e.g., from fuzzywuzzy or rapidfuzz) are commonly used for string matching. However, they have several limitations for this task:

| **Method**                   | **Limitation**                                                                |
| ---------------------------- | ----------------------------------------------------------------------------- |
| **Character-based Matching** | Based on spelling, not actual meaning (semantic understanding is missing)     |
| **No Context Awareness**     | Fails when words are rearranged (e.g., “NPK fertilizer” vs. “fertilizer NPK”) |
| **Typos/Synonyms Ignored**   | Cannot handle synonyms, misspellings, or abbreviations well                   |
| **High Time Complexity**     | Comparison time grows rapidly with data size (not suitable for many-to-many)  |
| **Poor Scalability**         | Slow with large vocabularies; computationally expensive at scale              |
| **Many-to-Many Matching**    | Becomes very expensive when comparing each POS entry to all catalog entries   |

➡️ Conclusion: Fuzzy matching is not suitable for large-scale, high-variance text matching where semantic meaning and speed are important.

✅ **Why Semantic Matching with BERT + FAISS?**

To overcome fuzzy matching limitations, we use a semantic similarity approach powered by Sentence-BERT and FAISS:

**Core Idea:**

Use a **BERT-based model** to convert text into semantic embeddings (vectors), then use **FAISS** to efficiently search for the most similar entries using **cosine similarity**.

**Proposed Method:**

* Convert both POS names and catalog entries into vector embeddings using a semantic model (BERT-based)
* Use FAISS for fast approximate nearest neighbor search
* Compute cosine similarity between vectors
* Match product if similarity exceeds a defined threshold (e.g. >0.8)
* If no match found → propose it as a new product

⚙️ **Proposed Methodology**

1. **Text Embedding:**

* Convert both POS product names and catalog SKUs into dense vector embeddings using Sentence-BERT (all-MiniLM-L6-v2).
* These embeddings capture semantic meaning — not just surface-level text.

2. **Indexing with FAISS:**

* Store catalog embeddings in a FAISS index (Facebook AI Similarity Search).
* Enables fast and scalable nearest-neighbor search even with large datasets.

3. **Similarity Search:**

* For each POS product name, retrieve top similar catalog entries based on cosine similarity.
* If the highest similarity exceeds a set threshold (e.g., 0.8), assign the corresponding SKU.

4. **Fallback Strategy:**

* If similarity is too low, flag the POS product name for manual review or proposal of a new SKU.


**Architecture Summary:**

```mermaid
flowchart TD
    A[POS Product Name] --> B[Text Cleaning]
    B --> C[Sentence-BERT Embedding]
    C --> D[FAISS Nearest Neighbor Search]
    D --> E{Similarity > Threshold?}
    E -- Yes --> F[Assign Existing SKU]
    E -- No --> G[Flag as New Product]
```


6. **🧹 Data Overview, Cleaning & Preprocessing**

* Lowercasing, punctuation removal, standardizing percent signs, removing non-alphanumeric characters
* Convert formula to consistent format (e.g., “46%” → “46”)
* Examples:

"Urea PIM 46%" → "urea pim 46"

"NPK 15-15-15 Kebomas" → "npk kebomas 15 15 15"



7. **🔍 Exploratory Data Analysis (EDA)**

* Word frequency analysis in POS data (e.g., most common fertilizers/formulas)
* Detect duplicate products across POS names
* Assess naming inconsistencies and spelling variation
* Evaluate how many unique POS names map to each SKU

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Load both Excel files
catalog = pd.read_excel(r"Product Catalog.xlsx")
pos = pd.read_excel(r"Product Name from PoS Transactions.xlsx")

In [2]:
# Show first few rows
display(catalog.head())
display(pos.head())

,Product SKU,Brand,Type,Formula
0,Urea Petro,PIHC,Urea,NaN
1,Urea PIM,PIHC,Urea,NaN
2,Urea Nitrea,PIHC,Urea,NaN
3,Urea Daun Buah,PIHC,Urea,NaN
4,Urea Pusri,PIHC,Urea,NaN


,Product Name
0,Pupuk Urea N 46%
1,Pupuk Amonium Sulfat ZA
2,Pupuk Super Fosfat SP-36
3,Pupuk NPK Phonska
4,Pupuk NPK Formula Khusus


### **Data Cleansing & Pre-processing**

In [3]:
import re

# Simple cleaning function
def clean_text(text):
    if not isinstance(text, str):
        text = ''  # or str(text) if you want to convert floats to strings
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s\-+]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Apply cleaning
catalog['Cleaned_Fields'] = (catalog['Brand'].fillna('') + ' ' +
                                 catalog['Type'].fillna('') + ' ' +
                                 catalog['Formula'].fillna('')).apply(clean_text)

pos['Cleaned_Product_Name'] = pos['Product Name'].apply(clean_text)


8. **🧠 Feature Engineering**
   
* **Construct semantic features** for catalog products by combining:

```
Brand + Type + Formula → concatenated text → vector embedding
```

* **Transform POS product names** into semantic vectors using Sentence-BERT

* **Compute similarity** using cosine distance in FAISS for matching

* **Apply dynamic thresholding** to determine if a match is strong enough (e.g., > 0.8)

  note we can adjest the threshold.

> (Optional) Extract "formula" patterns (e.g., 15-15-15) from product names using RegEx for downstream validation or fine-tuning

### **Embedding with BERT**

Feed the cleaned text into the Sentence-BERT model to convert it into vector embeddings that capture the semantic meaning.

In [4]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run script server
troubleshoot trust

Ju

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')

# we can adjust with paraphrase-mpnet-base-v2, all-mpnet-base-v2, all-MiniLM-L6-v2
#embeddings_product_name = model.encode(pos['Cleaned_Product_Name'], convert_to_tensor=True)
#embeddings_catalog = model.encode(catalog['Cleaned_Fields'], convert_to_tensor=True)

In [ ]:
embeddings_product_name = model.encode(pos['Cleaned_Product_Name'].tolist(), batch_size=64, convert_to_tensor=True)
embeddings_catalog = model.encode(catalog['Cleaned_Fields'].tolist(), batch_size=64, convert_to_tensor=True)


In [ ]:
print(embeddings_product_name.shape)
print(embeddings_product_name[0])     # vector for 'urea petro'


In [ ]:
print(embeddings_catalog.shape)
print(embeddings_catalog[0])     # vector for 'urea petro'


### **Build FAISS Index for Catalog Embeddings**

FAISS is a library for fast similarity search. You’ll index the catalog embeddings so you can quickly find nearest neighbors for each POS product embedding.

In [ ]:
import faiss
import numpy as np

# Assume you already have:
# embeddings_catalog, embeddings_pos (both are PyTorch tensors from Sentence-BERT)

# Convert PyTorch tensors to numpy arrays (FAISS works with numpy)
catalog_vectors = embeddings_catalog.cpu().numpy().astype('float32')
pos_vectors = embeddings_product_name.cpu().numpy().astype('float32')

# Build FAISS index
d = catalog_vectors.shape[1]  # dimension of embeddings
index = faiss.IndexFlatIP(d)  # Using Inner Product which works as cosine similarity if vectors are normalized

# Normalize vectors to unit length for cosine similarity
faiss.normalize_L2(catalog_vectors)
faiss.normalize_L2(pos_vectors)

# Add catalog vectors to the index
index.add(catalog_vectors)

# Search nearest neighbors for POS vectors
k = 1  # number of nearest neighbors to retrieve
distances, indices = index.search(pos_vectors, k)

# distances contain similarity scores, indices contain catalog indices of nearest neighbor
# Set a similarity threshold for matching
threshold = 0.5

# Create a match result list
matches = []
for i, (dist, idx) in enumerate(zip(distances, indices)):
    sim_score = dist[0]
    catalog_idx = idx[0]
    if sim_score >= threshold:
        matched_sku = catalog.iloc[catalog_idx]['Product SKU']
    else:
        matched_sku = None  # or flag as new product
    matches.append({'POS_Product': pos.iloc[i]['Product Name'],
                    'Matched_SKU': matched_sku,
                    'Similarity': sim_score})

# Convert matches to DataFrame for easy viewing
import pandas as pd
match_df = pd.DataFrame(matches)
print(match_df)

### **Histogram similarity scores**

In [ ]:
import matplotlib.pyplot as plt

plt.hist(match_df['Similarity'], bins=50, color='skyblue')
plt.axvline(x=0.8, color='red', linestyle='--', label='Threshold 0.5')
plt.xlabel('Similarity Score')
plt.ylabel('Number of POS Products')
plt.title('Distribution of Similarity Scores')
plt.legend()
plt.show()


### **Bar plot: Count match vs no match**

In [ ]:
match_counts = match_df['Matched_SKU'].notna().value_counts()

plt.bar(['Matched', 'Not Matched'], match_counts, color=['green', 'orange'])
plt.title('Number of Matched vs Not Matched POS Products')
plt.show()


### **Boxplot similarity matched/unmatched**

In [ ]:
import seaborn as sns

match_df['Matched'] = match_df['Matched_SKU'].notna()

sns.boxplot(x='Matched', y='Similarity', data=match_df)
plt.title('Similarity Score Distribution by Match Status')
plt.show()


9. **🧪 Model Training & Evaluation**
    
* Evaluation approach:
- Hit rate (matched vs unmatched)


### **Hit Rate / Match Rate**

In [ ]:
threshold = 0.5
match_df['Matched'] = match_df['Similarity'] >= threshold
hit_rate = match_df['Matched'].mean()
print(f"Hit Rate: {hit_rate:.2%}")



In [ ]:
# Assuming match_df already has 'Similarity' and 'Matched' columns (Matched = Similarity >= threshold)

sns.countplot(x='Matched', data=match_df)
plt.title(f'Hit Rate at Similarity Threshold {threshold}')
plt.xlabel('Matched (Similarity >= threshold)')
plt.ylabel('Count of POS Products')
plt.show()


10. **📌 Insight**
    
* **Hit Rate:**
The proportion of POS products matched above the similarity threshold indicates how many products in your POS system have a close counterpart in the catalog. For example, if hit rate is low (<50%), many POS items are either new products or naming differences are too large for the current model to detect.

*  **Similarity Distribution:**
Visualizing similarity scores for matched vs unmatched products shows how confident the model is on positive matches. Overlapping or low average similarity scores for matches may indicate noisy data or need for better preprocessing.

* **Threshold Sensitivity:**
Higher thresholds improve precision but lower recall, causing many products to remain unmatched. Conversely, lower thresholds increase matches but risk more false positives.

12. **📚 References & Tools**
    
* FAISS - Facebook AI Similarity Search
* https://www.geeksforgeeks.org/nlp/explanation-of-bert-model-nlp/
* https://en.wikipedia.org/wiki/BERT_(language_model)
* https://h2o.ai/wiki/bert/#:~:text=BERT%20is%20a%20neural%2Dnetwork,of%20words%20in%20search%20queries.
* https://huggingface.co/docs/transformers/en/model_doc/bert
* https://towardsdatascience.com/a-complete-guide-to-bert-with-code-9f87602e4a11/
* https://www.datacamp.com/blog/what-is-bert-an-intro-to-bert-models
* https://faiss.ai/index.html
* https://www.datacamp.com/blog/faiss-facebook-ai-similarity-search
* https://medium.com/@nishthakukreti.01/faiss-the-low-level-inner-workings-of-vector-search-and-optimization-cd642dec3518

**Python Libraries: pandas, numpy, sentence-transformers, faiss-cpu, scikit-learn, seaborn**

